In [29]:
import pandas as pd
import requests as r
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from bs4 import BeautifulSoup
from matplotlib.pyplot import xlabel
import os


In [30]:
plot_folder = '/Users/eddie/cs163/src/Plots'
data_folder = '/Users/eddie/cs163/src/Data'

# Data Collection
## CPI Data Import

In [158]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

data = {
    "seriesid": ["CUSR0000SAF"],
    "startyear": "2005",
    "endyear": "2024",
    "catalog": True,
    "calculations": False,
    "annualaverage": False,
    "aspects": False,
    "registrationkey": "5974f3d1e0ac4efdad23d2e1b3e4b4e3"
}

html = r.post(url, json=data)
html = html.json()
data = html['Results']['series'][0]['data']
CPI_data = pd.DataFrame(data, columns=['year', 'periodName', 'value'])
month_to_num = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

CPI_data.periodName = CPI_data.periodName.apply(lambda x: month_to_num[x])
CPI_data.rename(columns={'year': 'year', 'periodName': 'month', 'value': 'CPI'}, inplace=True)
CPI_data['date'] = pd.to_datetime(CPI_data[['year', 'month']].assign(day=1))
CPI_data.sort_values(by='date', ascending=True, inplace=True)
CPI_data['CPI'] = pd.to_numeric(CPI_data['CPI'], errors='coerce')

In [159]:
# Calculate percentage rate of change of CPI
def rate_of_change(data):
    rate = np.zeros(len(data))
    for time_frame in range(1, len(data)):
        tmp = float(data.iloc[time_frame, 2])
        prev = float(data.iloc[time_frame - 1, 2])
        rate[time_frame] = (tmp - prev) * 100 / (tmp + prev)
    return rate

CPI_data['rate_of_change'] = rate_of_change(CPI_data)

In [160]:
CPI_data.to_csv(os.path.join(data_folder, 'CPI_Processed_data.csv'))

In [161]:
fig = px.line(CPI_data, x='date', y='CPI', title='Timeseries of CPI Monthly', labels={'date': 'Year', 'rate_of_change': 'Rate of Change (%)'})

fig.write_image(os.path.join(plot_folder, 'CPI_data.png'))
fig.show()

In [162]:
fig = px.line(CPI_data, x='date', y='rate_of_change', title='Rate of Change of CPI Monthly', labels={'date': 'Year', 'rate_of_change': 'Rate of Change (%)'})
fig.write_image(os.path.join(plot_folder, 'CPI ROC.png'))
fig.show()

In [163]:
CPI_data.describe()

,month,CPI,date,rate_of_change
count,236.000000,236.000000,236,236.000000
mean,6.432203,245.285195,2014-10-16 07:43:43.728813568,0.116584
min,1.000000,189.100000,2005-01-01 00:00:00,-0.187437
25%,3.000000,219.073750,2009-11-23 12:00:00,0.041235
50%,6.000000,244.803000,2014-10-16 12:00:00,0.089714
75%,9.000000,258.703250,2019-09-08 12:00:00,0.160337
max,12.000000,327.849000,2024-08-01 00:00:00,0.648040
std,3.445574,35.922101,NaN,0.127634


## Food Price Collection

In [164]:
df_food = pd.read_csv('ap.data.3.Food', sep='\t', low_memory=False)
columns = []
for col in df_food.columns:
    columns.append(col.strip())
    
df_food = df_food.apply(lambda x: x.str.rstrip() if x.dtype == "object" else x)
df_food.columns = columns

In [165]:
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145331 entries, 0 to 145330
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   series_id       145331 non-null  object 
 1   year            145331 non-null  int64  
 2   period          145331 non-null  object 
 3   value           145331 non-null  object 
 4   footnote_codes  0 non-null       float64
dtypes: float64(1), int64(1), object(3)
memory usage: 5.5+ MB


In [166]:
df_food.iloc[:, 0].nunique()

672

In [167]:
food_items_series = df_food['series_id'].unique().tolist()
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
series = []
step = 48
for page in range(1, int(len(food_items_series) / step)):
        data1 = {
            "seriesid": food_items_series[page * step : (page + 1) * step],
            "startyear": "2005",
            "endyear": "2024",
            "catalog": True,
            "calculations": False,
            "annualaverage": False,
            "aspects": False,
            "registrationkey": "5974f3d1e0ac4efdad23d2e1b3e4b4e3"
        }
        html = r.post(url, json=data1)
        html = html.json()
        series.append(html['Results']['series'])

In [168]:
item_list = []
for item in series[0]:
    item_name = item['catalog']['measure_data_type']
    for info in item['data']:
        year = info['year']
        month = info['periodName']
        value = info['value']
        entry = {'item': item_name, 'year': year, 'month': month, 'value': value}
        item_list.append(entry)
df_item = pd.DataFrame(item_list)
df_item.month = df_item.month.apply(lambda x: month_to_num[x])
df_item['date'] = pd.to_datetime(df_item[['year', 'month']].assign(day=1))
df_item['value'] = pd.to_numeric(df_item['value'], errors='coerce').fillna(0)
df_item.drop(columns=['year', 'month'], inplace=True)
df_item.rename(columns={'value': 'Item Price', 'date': 'date'}, inplace=True)
df_item.to_csv(os.path.join(data_folder, 'item_price.csv'), index=False)
df_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5010 entries, 0 to 5009
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   item        5010 non-null   object        
 1   Item Price  5010 non-null   float64       
 2   date        5010 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 117.6+ KB


In [169]:
df_item.describe()

,Item Price,date
count,5010.000000,5010
mean,2.139684,2013-11-06 12:59:47.065868288
min,0.000000,2005-01-01 00:00:00
25%,1.259000,2009-03-01 00:00:00
50%,1.713500,2013-06-01 00:00:00
75%,2.704750,2018-04-01 00:00:00
max,6.357000,2024-08-01 00:00:00
std,1.294897,NaN


In [170]:
entry_availability = df_item.groupby('date')['item'].count()

fig = px.bar(entry_availability, title='Data Availability at different time periods ', labels={'date': 'Date', 'value': 'Availability'})
fig.write_image(os.path.join(plot_folder, 'item_availability.png'))
fig.show()


## Personal Consumption Expenditures 

In [174]:
from datetime import datetime
url = 'https://apps.bea.gov/api/data?&UserID=6E42AF4F-01C3-4ADB-9244-2DD195EFF66E&method=GetData&DataSetName=NIPA&TableName=T20805&Frequency=M&Year=2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024&ResultFormat=JSON'

html = r.get(url)
html = html.json()
food_pce = []
for table in html['BEAAPI']["Results"]["Data"]:
    if "Food" in table['LineDescription'] :
        food_pce.append(table)

df_pce = pd.DataFrame(food_pce, columns=['LineDescription', 'TimePeriod', 'DataValue'])
time_data = []
for time in df_pce['TimePeriod']:
    tmp = time.split('M')
    year = int(tmp[0])
    month = int(tmp[1])
    time_data.append(datetime(year, month, 1))

df_pce['TimePeriod'] = time_data
df_pce['DataValue'] = pd.to_numeric(df_pce['DataValue'].str.replace(',',''), errors='coerce')
df_pce.rename(columns={'DataValue': 'PCE', 'LineDescription': 'PCE Category', 'TimePeriod': 'date'}, inplace=True)
df_pce.to_csv(os.path.join(data_folder, 'pce.csv'), index=False)

In [175]:
df_pce.describe()

,date,PCE
count,470,4.700000e+02
mean,2014-10-01 02:27:03.829787136,9.053343e+05
min,2005-01-01 00:00:00,4.971490e+05
25%,2009-11-08 12:00:00,7.123938e+05
50%,2014-10-01 00:00:00,8.552145e+05
75%,2019-08-24 06:00:00,1.040535e+06
max,2024-07-01 00:00:00,1.487246e+06
std,NaN,2.526043e+05


In [176]:
df_pce.sort_values(by='date', ascending=True, inplace=True)
fig = px.line(df_pce, x='date', y='PCE', color='PCE Category', title='PCE timeseries')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.write_image(os.path.join(plot_folder, 'pce.png'))
fig.show()

## USDA Price Received Index

In [ ]:
program = 'SURVEY'
sector = ['ANIMALS & PRODUCTS', 'CROPS']
group = ['ANIMAL TOTALS', 'CROP TOTALS']
commodity = ['ANIMAL TOTALS', 'CROP TOTALS']
category = 'INDEX FOR PRICE RECEIVED, 2011'


In [184]:
base_url = 'https://quickstats.nass.usda.gov/api/api_GET'

data_frames = []
for index in [0, 1]:

    params = {
        'key': 'E621BE9B-36D0-3FF0-9333-443D794A932D',
        'source_desc': program,
        'sector_desc': sector[index],
        'group_desc': group[index],
        'commodity_desc': commodity[index],
        'statisticcat_desc': 'INDEX FOR PRICE RECEIVED, 2011',
        'freq_desc' : 'MONTHLY'
    }
    
    # Make a request with these parameters
    response = r.get(base_url, params=params)
    
    json = response.json()
    data = json['data']
    
    df = pd.DataFrame(data, columns=['commodity_desc', 'reference_period_desc', 'year', 'Value'])
    df.rename(columns={'commodity_desc': 'Commodity', 'reference_period_desc': 'month abbreviation', 'year': 'year', 'Value': 'RPI'}, inplace=True)
    data_frames.append(df)

df_price_received = pd.concat(data_frames)

In [185]:
month_abbr_to_num = {
    'JAN': 1,
    'FEB': 2,
    'MAR': 3,
    'APR': 4,
    'MAY': 5,
    'JUN': 6,
    'JUL': 7,
    'AUG': 8,
    'SEP': 9,
    'OCT': 10,
    'NOV': 11,
    'DEC': 12
}
df_price_received['month'] = df_price_received['month abbreviation'].apply(lambda x: month_abbr_to_num[x]) 

In [186]:
df_price_received['RPI'] = pd.to_numeric(df_price_received['RPI'])
df_price_received['date'] = pd.to_datetime(df_price_received[['month', 'year']].assign(day=1))
df_price_received = df_price_received[df_price_received['date'] >= '2005-01-01']

In [187]:
df_price_received.drop(['month', 'year', 'month abbreviation'], axis=1, inplace=True)

In [188]:
df_price_received.sort_values('date', ascending=False, inplace=True)
df_price_received.to_csv(os.path.join(data_folder, 'RPI.csv'))

In [194]:
fig = px.line(df_price_received, x='date', y='RPI', color='Commodity', title='RPI of each Commodity timeseries')
fig.write_image(os.path.join(plot_folder, 'RPI Timeseries.png'))
fig.show()

### Combine Dataset

In [31]:
df_pce = pd.read_csv(os.path.join(data_folder, 'pce.csv'))
df_price_received = pd.read_csv(os.path.join(data_folder, 'RPI.csv'))
df_item_price = pd.read_csv(os.path.join(data_folder, 'item_price.csv'))
CPI_data = pd.read_csv(os.path.join(data_folder, 'CPI_Processed_data.csv'))

In [32]:
df_price_received = df_price_received.pivot_table(values='RPI', index='date', columns='Commodity')
df_pce = df_pce.pivot_table(values='PCE', index='date', columns='PCE Category')
df_item_price = df_item_price.pivot_table(values='Item Price', index='date', columns='item')
CPI_data

,Unnamed: 0,year,month,CPI,date,rate_of_change
0,235,2005,1,189.100,2005-01-01,0.000000
1,234,2005,2,189.200,2005-02-01,0.026434
2,233,2005,3,189.600,2005-03-01,0.105597
3,232,2005,4,190.800,2005-04-01,0.315457
4,231,2005,5,191.000,2005-05-01,0.052383
...,...,...,...,...,...,...
231,4,2024,4,325.706,2024-04-01,0.009365
232,3,2024,5,326.153,2024-05-01,0.068573
233,2,2024,6,326.940,2024-06-01,0.120504
234,1,2024,7,327.488,2024-07-01,0.083737


In [33]:
step1 = CPI_data.merge(df_price_received, on='date', how='left')
step2 = step1.merge(df_item_price, on='date', how='left')
final = step2.merge(df_pce, on='date', how='left')
final.set_index('date', inplace=True)

In [34]:
final

,Unnamed: 0,year,month,CPI,rate_of_change,ANIMAL TOTALS,CROP TOTALS,"American processed cheese, per lb. (453.6 gm)","Apples, Red Delicious, per lb. (453.6 gm)","Bananas, per lb. (453.6 gm)",...,"Peaches, per lb. (453.6 gm)","Pears, Anjou, per lb. (453.6 gm)","Peppers, sweet, per lb. (453.6 gm)","Potatoes, white, per lb. (453.6 gm)","Strawberries, dry pint, per 12 oz. (340.2 gm)","Tomatoes, field grown, per lb. (453.6 gm)","Tuna, light, chunk, per lb. (453.6 gm)","Turkey, frozen, whole, per lb. (453.6 gm)",Food and beverages purchased for off-premises consumption,Food services and accommodations
date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,235,2005,1,189.100,0.000000,80.2,57.5,3.876,0.966,0.485,...,NaN,1.144,NaN,0.458,3.234,1.660,1.836,1.058,653299.0,515964.0
2005-02-01,234,2005,2,189.200,0.026434,78.7,59.6,3.873,0.974,0.502,...,1.757,1.101,NaN,0.448,2.346,1.428,1.806,1.063,655408.0,523076.0
2005-03-01,233,2005,3,189.600,0.105597,78.7,64.7,3.843,0.920,0.507,...,1.774,1.102,NaN,0.440,1.877,1.548,1.789,1.061,656803.0,521829.0
2005-04-01,232,2005,4,190.800,0.315457,79.6,67.4,3.840,0.869,0.503,...,NaN,1.156,NaN,0.450,1.526,1.710,1.889,1.058,664153.0,528354.0
2005-05-01,231,2005,5,191.000,0.052383,79.0,65.1,3.776,0.915,0.497,...,NaN,1.183,NaN,0.452,1.727,1.911,1.932,1.069,661961.0,529066.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,4,2024,4,325.706,0.009365,142.6,101.8,4.776,NaN,0.626,...,NaN,NaN,NaN,0.971,2.520,1.962,NaN,NaN,1467641.0,1422201.0
2024-05-01,3,2024,5,326.153,0.068573,142.9,103.8,4.819,NaN,0.622,...,NaN,NaN,NaN,0.964,2.259,1.859,NaN,NaN,1467947.0,1423153.0
2024-06-01,2,2024,6,326.940,0.120504,147.7,107.3,4.943,NaN,0.625,...,NaN,NaN,NaN,0.993,2.059,1.843,NaN,NaN,1475781.0,1425480.0


In [36]:
final.to_csv(os.path.join(data_folder, 'final.csv'))